<a href="https://colab.research.google.com/github/benedettoscala/ifttt-code-generator/blob/main/test_and_compare_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install -U bitsandbytes

In [3]:
!pip install rouge_scorer

ERROR: Could not find a version that satisfies the requirement rouge_scorer (from versions: none)
ERROR: No matching distribution found for rouge_scorer


In [4]:
!git clone https://github.com/benedettoscala/ifttt-code-generator
%cd ifttt-code-generator/
!git pull

Cloning into 'ifttt-code-generator'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 157 (from 1)
Receiving objects: 100% (159/159), 14.76 MiB | 41.54 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/ifttt-code-generator
Already up to date.


In [5]:
%cd ..

/content


In [6]:
    #load hugging face token
    from huggingface_hub import login
    login()

In [7]:
import pandas as pd
import torch
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from google.colab import drive
from peft import PeftModel

drive.mount('/content/drive')

# Load the dataset and split it
df = pd.read_csv("ifttt-code-generator/datasets/cleaned_and_combined.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Extract test set prompts
prompts = test_df["cleaned_description"].tolist()
actual_codes = test_df["filter_code"].tolist()

# Function to generate text with GPT-2
def generate_with_gpt2(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")
    generated_codes = []

    for prompt in prompts:
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
        output_ids = model.generate(input_ids, max_length=128, num_return_sequences=1)
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_codes.append(generated_text)

    del model
    del tokenizer
    torch.cuda.empty_cache()

    return generated_codes

# Function to generate text with BART
def generate_with_bart(model_path):
    generator = pipeline("text2text-generation", model=model_path, tokenizer=model_path)
    generated_codes = [generator(f"ifttt_prompt: {prompt}", max_length=128)[0]["generated_text"] for prompt in prompts]

    del generator
    torch.cuda.empty_cache()

    return generated_codes

# Function to generate text with Mistral

def generate_with_mistral(finetuned_model_path, basemodel_path):
    if not os.path.exists("./offload"):
        os.makedirs("./offload")




    print("Caricamento del modello fine-tunato...")
    bnb_config = BitsAndBytesConfig(
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False
    )

    model = AutoModelForCausalLM.from_pretrained(
        basemodel_path,
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
        device_map="auto",
        offload_folder="./offload"
    )

    model = PeftModel.from_pretrained(model, finetuned_model_path)
    tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)

    generated_codes = []
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_length=512,
            num_return_sequences=1,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=1,
        )
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        generated_codes.append(decoded_outputs[0])

    del model
    del tokenizer
    torch.cuda.empty_cache()

    return generated_codes

# Generate with Mistral
finetuned_model_path = "/content/drive/Shareddrives/NLPMODELS/mistral/checkpoint-20"
basemodel_path = "mistralai/Mistral-7B-Instruct-v0.2"
generated_codes_mistral = generate_with_mistral(finetuned_model_path, basemodel_path)


# Generate with BART
model_bart_path = "/content/drive/Shareddrives/NLPMODELS/nl2sql_bart_final/checkpoint-340"
generated_codes_bart = generate_with_bart(model_bart_path)


# Generate with GPT-2
model_gpt2_path = "/content/drive/Shareddrives/NLPMODELS/gpt2model/checkpoint-340"
generated_codes_gpt2 = generate_with_gpt2(model_gpt2_path)



# Create a DataFrame with results
results_df = pd.DataFrame({
    "Prompt": prompts,
    "Generated Code GPT-2": generated_codes_gpt2,
    "Generated Code BART": generated_codes_bart,
    "Generated Code Mistral": generated_codes_mistral,
    "Actual Code": actual_codes
})


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Caricamento del modello fine-tunato...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [8]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c6e2388e4256c737293942ba1dc9e9e7171c549d7da1eb3c419a8d806846e51e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [19]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score

def evaluate_generated_text(generated_codes, actual_codes):
    bleu_scores = []
    meteor_scores = []
    rouge_l_scores = []

    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

    for gen, ref in zip(generated_codes, actual_codes):
        gen_tokens = gen.split()
        ref_tokens = ref.split()

        # Calcolo BLEU (sentence-level)
        bleu = sentence_bleu([ref_tokens], gen_tokens)

        # Calcolo METEOR (sentence-level)
        meteor = single_meteor_score(ref_tokens, gen_tokens)

        # Calcolo ROUGE-L (f-measure)
        rouge_l = scorer.score(ref, gen)["rougeL"].fmeasure

        bleu_scores.append(bleu)
        meteor_scores.append(meteor)
        rouge_l_scores.append(rouge_l)

    # Media su tutte le frasi del dataset di test
    mean_bleu = sum(bleu_scores) / len(bleu_scores)
    mean_meteor = sum(meteor_scores) / len(meteor_scores)
    mean_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)

    return mean_bleu, mean_meteor, mean_rouge_l


In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
# Evaluate models
gpt2_scores = evaluate_generated_text(generated_codes_gpt2, actual_codes)
bart_scores = evaluate_generated_text(generated_codes_bart, actual_codes)
mistral_scores = evaluate_generated_text(generated_codes_mistral, actual_codes)

metrics_df = pd.DataFrame(
    {
        "Metric": ["BLEU", "METEOR", "ROUGE-L"],
        "GPT-2": gpt2_scores,
        "BART": bart_scores,
        "Mistral": mistral_scores
    }
)


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [21]:
metrics_df

,Metric,GPT-2,BART,Mistral
0,BLEU,0.106910,0.418573,0.062660
1,METEOR,0.374292,0.616489,0.284503
2,ROUGE-L,0.300527,0.712151,0.282811


In [15]:
import math

# ---- DEFINIAMO UNA FUNZIONE PER LA NORMALIZZAZIONE MINMAX ----
def minmax_normalize(csv_string):
    """
    csv_string è una stringa tipo:
    '0.123,0,1.23e-4, ...'
    Restituisce una lista di float normalizzati in [0,1] (MinMax).
    """
    # 1) trasformiamo la stringa in un vettore di float
    arr = [float(v) for v in csv_string.split(',')]
    mn = min(arr)
    mx = max(arr)
    if mx == mn:
        # se tutti i valori sono uguali, la normalizzazione min-max
        # andrebbe in divisione per zero. Mettiamo tutti a 0.
        norm = [0.0]*len(arr)
    else:
        norm = [(x - mn)/(mx - mn) for x in arr]
    return norm

# ---- ECCO I 3 RIGHE CHE HAI FORNITO NEL TUO CSV (BLEU, METEOR, ROUGE) ----

# 1) BLEU
gpt2_bleu_str = "7.817228731469945e-232,2.753730455889728e-155,0.20811215780505982,1.2992180285238283e-78,0.15702128402250726,8.619657720183654e-232,9.893133360884868e-232,0.3234043476847562,8.972141065609098e-232,0.14523626605098838,3.4437891216575777e-155,0,1.019432043742309e-231,0.478294125095073,0,1.0669733992029681e-231,1.3779825009092223e-78,0.1919704825961155,0.37565846334638286,1.2992180285238283e-78,1.5910769301604901e-78,0.4906921434897839,7.817228731469945e-232,1.185814295932958e-231,1.3162666718999167e-78,0,2.3744421761716724e-78,3.962327745598725e-155,1.0350003944289303e-231,0.16204461468762177,9.533966891110756e-232,0.2552634961498702,0.2911887740301409,0.15813859795767055,0.1572175759719851,2.366453678476723e-78,1.0901460182896916e-231,0.16879023702486318,1.2757520836838496e-78,8.972141065609098e-232,9.893133360884868e-232,0.07232031895361331,0.08601230531735271,0.5175988392176301,1.0638581151575014e-231,0.1508587474361693,0,0.08774216744205626,0.3666683450651074,1.2757520836838496e-78,2.65352324113073e-78,1.0377133938315695e-231,0.478294125095073,0.3968192834392067,0.12961166192752321,2.9363218606620623e-155,8.460552185460498e-232,0.35181590435495047"
bart_bleu_str = "0.28009266907237096,1.2213386697554703e-77,0.7366140739862848,0.36787944117144233,0.7348889200874658,2.6829510818488373e-79,0.355310106137518,0.36419953271771677,0.14352152945210084,1,6.545944537020445e-78,0,0.7099125225932005,3.648956622645728e-232,1,0.2531295708411999,5.669339150702353e-78,1,0.39765350473507033,0.36787944117144233,3.2683416997819595e-78,0.6587255217131245,1,4.921852225123671e-78,0.02802260025756584,0,0.8948393168143697,0.14931742814772206,1.2194411011180272e-233,0.20571757429957402,1,1,0.3195634113709065,0.5946035575013605,0.5983790078367693,0.5773502691896257,1.384292958842266e-231,1,1,0.14352152945210084,0.355310106137518,0.07274621764192092,0.5939240213416938,0.7693588470744102,3.648956622645728e-232,0.578733153991148,1.491668146240062e-154,0.8633400213704505,0.5615731785165538,1,3.045843896781981e-78,0,3.648956622645728e-232,0.4086103435932962,0.39765350473507033,1.2213386697554703e-77,1,0.794834366062997"
mistral_bleu_str = "7.114448003820407e-232,7.290245807398516e-232,1.643961915018504e-78,8.530853616266939e-232,2.5272837599348647e-78,1.0423153399406431e-231,0.027478397398956064,0.14508640899912412,7.00312622297947e-232,2.259351571061102e-155,4.017758378857155e-155,0,1.1268086493920282e-231,0.3130610448919025,9.309480961664223e-232,0.20653442117427914,1.7842265606563606e-78,8.477028509227734e-232,2.444438958570087e-155,3.3523603773965668e-155,9.947643045471176e-232,0.39560746658646495,9.857609866842017e-232,1.7563526234704057e-78,0.11700201243586499,0,2.1407019746909946e-78,1.039567908289092e-78,1.4427829315340329e-78,1.4597789197781286e-78,9.72161026064145e-232,0.28782893232478673,0.40203449793781326,1.0244914152188952e-231,2.1567203256506265e-78,1.1454162233972301e-231,1.1070900996592687e-231,0.2551765198658487,1.1105165272654406e-78,6.659801989051833e-232,0,0.04566615848271797,0.2701650163723732,0.07661972949118845,0.29197256936096194,0.10836164033218013,0,0,0.04280622976493489,1.1728278695261032e-78,2.780996055569847e-78,8.90735602648238e-232,0.3836927089871836,1.7745179415358185e-78,0.10903316620080986,7.03404585427606e-232,8.259885936748448e-232,0.15616086104328353"

# 2) METEOR
gpt2_meteor_str = "0.07075471698113207,0.14492753623188406,0.6281502016129032,0.30058262711864403,0.6126582278481014,0.0578034682080925,0.13793103448275862,0.29101144906545273,0.09646302250803858,0.702433628318584,0.13739545997610517,0,0.06830601092896174,0.7618306178057054,0.03968253968253969,0.0819672131147541,0.4292746113989638,0.6607407407407409,0.8562967051630435,0.30058262711864403,0.3073424301494478,0.8162768394428627,0.1617440225035162,0.20114942528735633,0.10144927536231885,0,0.5235748792270531,0.1286764705882353,0.08264462809917356,0.6092057761732852,0.14563106796116504,0.7217890405024907,0.6079985917270607,0.6160910087719298,0.5580357142857143,0.41333333333333344,0.18115942028985504,0.5694158075601373,0.38419394603478974,0.09646302250803858,0.13793103448275862,0.13043478260869568,0.619768644785103,0.8044025623640423,0.06983240223463687,0.6274592256630522,0,0.5813754073057795,0.6599999999999999,0.38419394603478974,0.6850615447900515,0.0847457627118644,0.7618306178057054,0.8686389641674195,0.5079114581170315,0.29296875000000006,0.05102040816326531,0.8363782051282052"
bart_meteor_str = "0.5386178861788617,0.9814814814814815,0.8698442145924928,0.5255485652907779,0.9139118457300276,0.12830284552845528,0.4523364485981308,0.3954560773321716,0.37055335968379444,0.9993141289437586,0.7010582010582012,0,0.8838561022744165,0.07575757575757576,0.9990234375,0.44910179640718556,0.7937223245424977,0.9996243425995492,0.4883227176220807,0.5255485652907779,0.4059921841076856,0.785223299427845,0.9996243425995492,0.45884773662551437,0.17386642156862747,0,0.9084673899488716,0.4056437389770724,0.04672897196261682,0.42240587695133147,0.9985422740524781,0.9999142661179699,0.657827184128089,0.864795918367347,0.9814814814814815,0.8420118343195266,0.21929824561403505,0.999968,0.9996243425995492,0.37055335968379444,0.4523364485981308,0.1388888888888889,0.7729447711287906,0.826300592179318,0.07575757575757576,0.840561224489796,0.9375,0.8880208333333333,0.6818871359223301,0.9996243425995492,0.5703955135773318,0,0.07575757575757576,0.5499999999999999,0.4883227176220807,0.9814814814814815,0.999898229187869,0.8444636678200692"
mistral_meteor_str = "0.06276150627615064,0.07575757575757576,0.2904099598895661,0.10869565217391305,0.5361305361305362,0.056285178236397754,0.30988432239255154,0.19736842105263158,0.08474576271186442,0.12738853503184713,0.05780346820809248,0,0.08426966292134833,0.7551473809374805,0.1724137931034483,0.5997570990101023,0.39329805996472667,0.1226993865030675,0.07177033492822966,0.24365079365079362,0.11695906432748539,0.778125,0.23554104477611942,0.2854713313896987,0.23148148148148145,0,0.43634792626728114,0.22566282129123658,0.3720818185800009,0.34879406307977734,0.15000000000000002,0.7574932391826924,0.786178107606679,0.1470588235294118,0.4307116104868915,0.1497005988023952,0.12396694214876032,0.5628896222955629,0.27332621082621084,0.06082725060827251,0,0.13404825737265413,0.8326155612977318,0.32478002487676794,0.7191967696169377,0.4035087719298246,0,0,0.15873015873015872,0.2496644295302014,0.4821621621621622,0.0746268656716418,0.7597784592277365,0.32206632653061223,0.5264290633608816,0.06944444444444445,0.06696428571428573,0.5543177969877908"

# 3) ROUGE-L
gpt2_rouge_str = "0.05333333333333334,0.2545454545454546,0.43809523809523804,0.11267605633802816,0.34782608695652173,0.17777777777777778,0.1791044776119403,0.48275862068965514,0.05714285714285715,0.3333333333333333,0.11320754716981131,0.08955223880597013,0.1075268817204301,0.6538461538461539,0.056338028169014086,0.10869565217391304,0.2535211267605634,0.5753424657534247,0.8562967051630435,0.11267605633802816,0.23684210526315788,0.673469387755102,0.18666666666666665,0.23529411764705882,0.35294117647058826,0.07692307692307691,0.3529411764705882,0.18000000000000002,0.08791208791208792,0.46575342465753433,0.14814814814814814,0.425,0.3870967741935484,0.3870967741935484,0.4285714285714286,0.4313725490196079,0.23529411764705882,0.4096385542168674,0.2,0.05714285714285715,0.1791044776119403,0.253968253968254,0.39999999999999997,0.6804123711340206,0.09195402298850575,0.39344262295081966,0.06779661016949153,0.2857142857142857,0.6206896551724138,0.2,0.4680851063829786,0.23529411764705882,0.6538461538461539,0.626865671641791,0.32,0.3103448275862069,0.03125,0.5526315789473685"
bart_rouge_str = "0.6842105263157895,1,0.7647058823529412,0.6666666666666666,0.8571428571428571,0.26229508196721313,0.6206896551724138,0.7164179104477612,0.5483870967741935,1,0.8181818181818182,0.16,0.9047619047619048,0.21052631578947364,1,0.6268656716417911,0.9285714285714286,1,0.5161290322580646,0.6666666666666666,0.5714285714285715,0.8857142857142858,1,0.6923076923076923,0.4482758620689655,0,0.9714285714285714,0.40625,0.22222222222222218,0.7000000000000001,1,1,0.5396825396825397,1,1,0.9230769230769231,0.4,1,1,0.5483870967741935,0.6206896551724138,0.5652173913043479,0.888888888888889,0.8307692307692307,0.21052631578947364,0.9230769230769231,1,0.9375,0.8292682926829269,1,0.8148148148148148,0.25,0.21052631578947364,0.5625,0.5,1,1,0.9"
mistral_rouge_str = "0.057803468208092484,0.16666666666666666,0.2790697674418604,0.11688311688311688,0.2553191489361702,0.15384615384615383,0.189873417721519,0.30927835051546393,0.0948905109489051,0.15686274509803924,0.20833333333333331,0.14285714285714288,0.18604651162790697,0.6610169491525423,0.17857142857142858,0.5504587155963303,0.34615384615384615,0.12195121951219513,0.12222222222222222,0.2608695652173913,0.15873015873015875,0.6181818181818182,0.3934426229508197,0.30434782608695654,0.26506024096385544,0.0975609756097561,0.39999999999999997,0.1320754716981132,0.3916083916083916,0.2716049382716049,0.1568627450980392,0.5151515151515151,0.6190476190476191,0.339622641509434,0.2807017543859649,0.2,0.3529411764705882,0.41999999999999993,0.17391304347826086,0.032388663967611336,0.09448818897637795,0.19148936170212766,0.40625,0.42592592592592593,0.6608695652173914,0.3728813559322034,0.045454545454545456,0.10169491525423728,0.21782178217821782,0.19047619047619047,0.4186046511627907,0.27586206896551724,0.6101694915254238,0.4516129032258065,0.5217391304347826,0.163265306122449,0.021505376344086023,0.5507246376811594"

# ---- CREIAMO LE LISTE NORMALIZZATE ----

# BLEU
norm_gpt2_bleu    = minmax_normalize(gpt2_bleu_str)
norm_bart_bleu    = minmax_normalize(bart_bleu_str)
norm_mistral_bleu = minmax_normalize(mistral_bleu_str)

# METEOR
norm_gpt2_meteor    = minmax_normalize(gpt2_meteor_str)
norm_bart_meteor    = minmax_normalize(bart_meteor_str)
norm_mistral_meteor = minmax_normalize(mistral_meteor_str)

# ROUGE-L
norm_gpt2_rouge    = minmax_normalize(gpt2_rouge_str)
norm_bart_rouge    = minmax_normalize(bart_rouge_str)
norm_mistral_rouge = minmax_normalize(mistral_rouge_str)

# ---- ORA CREIAMO UN PICCOLO CSV FINALE SIMILE AL TUO, MA COI VALORI NORMALIZZATI ----

def list_to_str(arr):
    # Trasforma la lista di float in una stringa CSV, con un minimo di formattazione
    return ",".join(f"{x:.6g}" for x in arr)

print("index,Metric,GPT-2,BART,Mistral")

# Riga 0: BLEU
print(f"0,BLEU,\"{list_to_str(norm_gpt2_bleu)}\",\"{list_to_str(norm_bart_bleu)}\",\"{list_to_str(norm_mistral_bleu)}\"")
# Riga 1: METEOR
print(f"1,METEOR,\"{list_to_str(norm_gpt2_meteor)}\",\"{list_to_str(norm_bart_meteor)}\",\"{list_to_str(norm_mistral_meteor)}\"")
# Riga 2: ROUGE-L
print(f"2,ROUGE-L,\"{list_to_str(norm_gpt2_rouge)}\",\"{list_to_str(norm_bart_rouge)}\",\"{list_to_str(norm_mistral_rouge)}\"")


index,Metric,GPT-2,BART,Mistral


NameError: name 'avg' is not defined